#Notebook Description
**Author**: Slawomir Drzymala

**Description:**   
This notebook is getting the data from the enriched layer of the data lake and prepares the database tables inside Databricks

#Set up connection to data lake on Azure

**Things to be noticed:**   
* **sensitive data alert** - please note that this is not recommended to store any key or any other sensitve data in the notebooks, this is just to make the code more simple for the demo. For real work please use Azure KeyVault or databricks secrets.
* **multiple ways to connect to Azure data lake** - there are multiple options to connect to the Azure data lake, we can use the access key or the service principal, we can also mount the storage account so the storage account will be visible in many notebooks, please see link below for mode details

In [0]:
#vide https://docs.databricks.com/_static/notebooks/data-import/azure-data-lake-store.html
#vide https://docs.microsoft.com/en-us/azure/databricks/data/data-sources/azure/adls-gen2/azure-datalake-gen2-get-started
spark.conf.set(
  "fs.azure.account.key.sdsalearnsthnew.dfs.core.windows.net", 
  "RJMELuc9ffZPf5D0gwcbxJp+hWTkQuW8lmWa1DRFSF59aDiatDsMJ6X/yC/dHZtB7kdGl3cJIrYry++6EnCb5g==" 
)


#Create database

**Things to be noticed:**   
* **Databricks database** - An Azure Databricks database is a collection of tables. An Azure Databricks table is a collection of structured data. You can cache, filter, and perform any operations supported by Apache Spark DataFrames on Azure Databricks tables. You can query tables with Spark APIs and Spark SQL.

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS RadioPlaylistAnalysis")
spark.sql("USE RadioPlaylistAnalysis")

Out[54]: DataFrame[]

#Create table - song

**Things to be noticed:**   
* **Databricks tables** - table is a collection of structured data.
   * createOrReplaceTempView (in older version of the API there was registerTempTable()) - function to build a temporaty table to run SQL commands on our DataFrame at scale! A point to remember is that the lifetime of this temp table is tied to the session. It creates an in-memory table that is scoped to the cluster in which it was created. The data is stored using Hive's highly-optimized, in-memory columnar format.
   * saveAsTable() - You can also check out saveAsTable() which creates a permanent, physical table stored in S3 using the Parquet format. This table is accessible to all clusters. The table metadata including the location of the file(s) is stored within the Hive metastore. Please note that you can specify the path yourself to save the data to the data lake storage

In [0]:
from delta.tables import *

# read delta table and convert to dataframe
input_file = "abfss://learnsthnew@sdsalearnsthnew.dfs.core.windows.net/enriched/songs.parquet"
songs_delta_table = DeltaTable.forPath(spark, input_file)
df_songs = songs_delta_table.toDF()

# save to dedicated file
output_path = "abfss://learnsthnew@sdsalearnsthnew.dfs.core.windows.net/enriched/RadioPlaylistAnalysis/songs.parquet"
df_songs.write.mode('overwrite').parquet(output_path)

# create unmanaged table
df_songs.write.mode('overwrite').saveAsTable("RadioPlaylistAnalysis.song")


#Create table - playlist

**Things to be noticed:**   
* **Databricks tables** - table is a collection of structured data.
   * createOrReplaceTempView (in older version of the API there was registerTempTable()) - function to build a temporaty table to run SQL commands on our DataFrame at scale! A point to remember is that the lifetime of this temp table is tied to the session. It creates an in-memory table that is scoped to the cluster in which it was created. The data is stored using Hive's highly-optimized, in-memory columnar format.
   * saveAsTable() - You can also check out saveAsTable() which creates a permanent, physical table stored in S3 using the Parquet format. This table is accessible to all clusters. The table metadata including the location of the file(s) is stored within the Hive metastore. Please note that you can specify the path yourself to save the data to the data lake storage

In [0]:
from delta.tables import *

# read delta table and convert to dataframe
base_path = "abfss://learnsthnew@sdsalearnsthnew.dfs.core.windows.net/enriched/"
file_path = f"abfss://learnsthnew@sdsalearnsthnew.dfs.core.windows.net/enriched/playlist/radio_name=*/year=*/*.parquet"
df_playlist = spark.read.option("basePath", base_path) \
                        .option('encoding', 'UTF-8') \
                        .parquet(file_path, multiLine=True)

# save to dedicated file
output_path = "abfss://learnsthnew@sdsalearnsthnew.dfs.core.windows.net/enriched/RadioPlaylistAnalysis/playlist/"
df_playlist.write.mode('overwrite').partitionBy("radio_name", "year").parquet(output_path)

# create unmanaged table
df_playlist.write.mode('overwrite').saveAsTable("RadioPlaylistAnalysis.playlist")


#Query table using spark sql

In [0]:
%sql

select * 
from RadioPlaylistAnalysis.song
where is_track_downloaded == 'True'
limit 10

artist_and_title,artist,title,cnt,track_id,track_href,track_is_local,track_is_playable,track_name,track_popularity,track_track_number,track_type,track_uri,track_duration_ms,track_disc_number,track_explicit,track_external_ids_isrc,album_album_type,album_href,album_id,album_name,album_release_date,album_release_date_precision,album_total_tracks,album_type,album_uri,is_track_downloaded,audio_features_danceability,audio_features_energy,audio_features_key,audio_features_loudness,audio_features_mode,audio_features_speechiness,audio_features_acousticness,audio_features_instrumentalness,audio_features_liveness,audio_features_valence,audio_features_tempo,is_audio_features_downloaded
Alan Walker / Sophia Somajo - Diamond Heart,Alan Walker / Sophia Somajo,Diamond Heart,620,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false
Alexandra Burke - Start Without You (Ft. Laza Morgan),Alexandra Burke,Start Without You (Ft. Laza Morgan),638,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false
Alle Farben / Janieck - Little Hollywood,Alle Farben / Janieck,Little Hollywood,561,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false
Armin Van Buuren - Not Giving Up On Love (Ft. Sophie Ellis,Armin Van Buuren,Not Giving Up On Love (Ft. Sophie Ellis,598,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false
Artur Gadowski - Ona Jest Ze Snu,Artur Gadowski,Ona Jest Ze Snu,3242,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false
Bednarek / Matheo - Talizman,Bednarek / Matheo,Talizman,525,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false
Biały - Ta Chwila,Biały,Ta Chwila,517,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false
Blade Loki - No Passaran,Blade Loki,No Passaran,747,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false
Blu Cantrell / Sean Paul - Breathe,Blu Cantrell / Sean Paul,Breathe,677,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false
Blue Cafe - Niewiele Mam,Blue Cafe,Niewiele Mam,584,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false


#Query table using spark sql

In [0]:
display(
  spark.sql("""

    select * 
    from RadioPlaylistAnalysis.song
    where is_track_downloaded == 'True'
    limit 10

  """)
)

artist_and_title,artist,title,cnt,track_id,track_href,track_is_local,track_is_playable,track_name,track_popularity,track_track_number,track_type,track_uri,track_duration_ms,track_disc_number,track_explicit,track_external_ids_isrc,album_album_type,album_href,album_id,album_name,album_release_date,album_release_date_precision,album_total_tracks,album_type,album_uri,is_track_downloaded,audio_features_danceability,audio_features_energy,audio_features_key,audio_features_loudness,audio_features_mode,audio_features_speechiness,audio_features_acousticness,audio_features_instrumentalness,audio_features_liveness,audio_features_valence,audio_features_tempo,is_audio_features_downloaded
Alan Walker / Sophia Somajo - Diamond Heart,Alan Walker / Sophia Somajo,Diamond Heart,620,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false
Alexandra Burke - Start Without You (Ft. Laza Morgan),Alexandra Burke,Start Without You (Ft. Laza Morgan),638,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false
Alle Farben / Janieck - Little Hollywood,Alle Farben / Janieck,Little Hollywood,561,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false
Armin Van Buuren - Not Giving Up On Love (Ft. Sophie Ellis,Armin Van Buuren,Not Giving Up On Love (Ft. Sophie Ellis,598,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false
Artur Gadowski - Ona Jest Ze Snu,Artur Gadowski,Ona Jest Ze Snu,3242,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false
Bednarek / Matheo - Talizman,Bednarek / Matheo,Talizman,525,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false
Biały - Ta Chwila,Biały,Ta Chwila,517,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false
Blade Loki - No Passaran,Blade Loki,No Passaran,747,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false
Blu Cantrell / Sean Paul - Breathe,Blu Cantrell / Sean Paul,Breathe,677,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false
Blue Cafe - Niewiele Mam,Blue Cafe,Niewiele Mam,584,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,true,#,#,#,#,#,#,#,#,#,#,#,false


#Create table usinig delta (optional)

In [0]:
spark.sql("""
    CREATE TABLE songs
    USING DELTA
    LOCATION 'abfss://learnsthnew@sdsalearnsthnew.dfs.core.windows.net/enriched/songs.parquet'
  """
)

Out[2]: DataFrame[]

#Drop table (optional)

In [0]:
spark.sql("drop table RadioPlaylistAnalysis.playlist")
spark.sql("drop table RadioPlaylistAnalysis.song")

Out[52]: DataFrame[]

In [0]:
%sql

select artist_and_title, count(*) as cntt
from RadioPlaylistAnalysis.song
group by artist_and_title
having count(*) > 1

artist_and_title,cntt


In [0]:
%sql

select count(*)
from RadioPlaylistAnalysis.playlist

count(1)
4614665
